## In this notebook, we will process and clean the raw data that is provided to train the model.

* To begine with, first lets load the data and display the first few lines to know what we are working with.

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor

In [2]:
file_path = 'C:/Users/ok/Desktop/kaggel_titanic_project/Kaggle_titanic/Data/train.csv'
file_path1 = 'C:/Users/ok/Desktop/kaggel_titanic_project/Kaggle_titanic/Data/test.csv'
data = pd.read_csv(file_path, index_col='PassengerId')
test = pd.read_csv(file_path1, index_col = 'PassengerId')
data.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
#data.isnull().sum()

In [4]:
#data.info()

* From the above cell, we  can see that most of the columons have data in them (are not empty). But we can see that the cabin has the most Nan values(more than half). And the age columon also has the second most highest Nan values.

* Hence since the cabin columon has more than 20% NaN value, i have decided to drop the cabin columon from the data, and impend the value for the age Nan values. And alos to for the two rows which has aNan value for the Embark columon

In [5]:
#now lets split the usable data into feature (X) and prediction (y), and split the data into training and validation
y = data.Survived
data.drop(['Survived'], axis=1, inplace=True)
X = data.copy()
#Before training the model with the training data lets calculate how accurate it is. 


In [6]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.7, test_size= 0.3, random_state=0)

In [7]:
#First lets split the data into numerical data and catagorical data(data with 'object')
num_col = [col for col in data.columns if data[col].dtype in ['int64', 'float64']]
obj_col = [col for col in data.columns if data[col].dtype in ['object']]

In [8]:
#Note that we need to convert the object data into numerical value for training the mcahine.
#Hence we need to use a good data (data with low cardinality)
cat_col = [cols for cols in obj_col if data[cols].nunique()< 10]
data_f = data[cat_col + num_col]
cols = cat_col + num_col

In [9]:
X_train = X_train[cols]
X_valid = X_valid[cols]

In [10]:

X_lable_train_num = X_train[num_col].copy()
X_lable_valid_num = X_valid[num_col].copy()

X_lable_train_cat = X_train[cat_col].copy()
X_lable_valid_cat = X_valid[cat_col].copy()

In [11]:
import numpy as np

In [12]:
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
X_lable_train_num_im = pd.DataFrame(imputer.fit_transform(X_lable_train_num))
X_lable_valid_num_im = pd.DataFrame(imputer.transform(X_lable_valid_num))

X_lable_train_num_im.columns = X_lable_train_num.columns
X_lable_valid_num_im.columns = X_lable_valid_num.columns

In [13]:
X_lable_valid_num_im.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 268 entries, 0 to 267
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Pclass  268 non-null    float64
 1   Age     268 non-null    float64
 2   SibSp   268 non-null    float64
 3   Parch   268 non-null    float64
 4   Fare    268 non-null    float64
dtypes: float64(5)
memory usage: 10.6 KB


In [25]:
imputer_cat = SimpleImputer(missing_values = np.NaN, strategy='most_frequent')

X_lable_train_cat_im = pd.DataFrame(imputer.fit_transform(X_lable_train_cat_en))
X_lable_valid_cat_im = pd.DataFrame(imputer.transform(X_lable_valid_cat_en))

X_lable_train_cat_im.columns = X_lable_train_cat.columns
X_lable_train_cat_im.columns = X_lable_train_cat.columns

In [21]:
from sklearn.preprocessing import OrdinalEncoder

ordinal = OrdinalEncoder()

X_lable_train_cat_en = ordinal.fit_transform(X_lable_train_cat)
X_lable_valid_cat_en = ordinal.transform(X_lable_valid_cat)



In [24]:
X_lable_valid_cat_en.info()

AttributeError: 'numpy.ndarray' object has no attribute 'info'

In [17]:
X_train_data = pd.concat([X_lable_train_num_im, X_lable_train_cat_im], axis=1)
X_valid_data = pd.concat([X_lable_valid_num_im, X_lable_valid_cat_im], axis=1)

NameError: name 'X_lable_train_cat_im' is not defined

In [26]:
X_valid_data.info()

NameError: name 'X_valid_data' is not defined

In [27]:
#X_train.info()
y_train.info()

<class 'pandas.core.series.Series'>
Index: 623 entries, 858 to 685
Series name: Survived
Non-Null Count  Dtype
--------------  -----
623 non-null    int64
dtypes: int64(1)
memory usage: 9.7 KB


In [21]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 623 entries, 858 to 685
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    623 non-null    int64  
 1   Name      623 non-null    object 
 2   Sex       623 non-null    object 
 3   Age       502 non-null    float64
 4   SibSp     623 non-null    int64  
 5   Parch     623 non-null    int64  
 6   Ticket    623 non-null    object 
 7   Fare      623 non-null    float64
 8   Cabin     152 non-null    object 
 9   Embarked  621 non-null    object 
dtypes: float64(2), int64(3), object(5)
memory usage: 53.5+ KB


In [28]:
#now lets preprocess the numical_datas ans the catagorical data by transformers
numerical_transformer = SimpleImputer(strategy='mean')
catagorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(transformers=[
    ('numerical', numerical_transformer, num_col),
    ('catagorical', catagorical_transformer, cat_col)
])

In [29]:
#now definr the model
model = RandomForestRegressor (random_state=0)

In [33]:
model.fit(X_train_data,y_train)

NameError: name 'X_train_data' is not defined

In [190]:
pre = model.predict(X_valid_data)

TypeError: Feature names are only supported if all input features have string names, but your input has ['int', 'str'] as feature name / column name types. If you want feature names to be stored and validated, you must convert them all to strings, by using X.columns = X.columns.astype(str) for example. Otherwise you can remove feature / column names from your input data, or convert them all to a non-string data type.

In [128]:
#finally we can define the pipeline
pipe = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('mode', model)
])

In [129]:
#now lets train the model over the train X ans y data then predict the valid X data
pipe.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('numerical', SimpleImputer(),
                                                  ['Pclass', 'Age', 'SibSp',
                                                   'Parch', 'Fare']),
                                                 ('catagorical',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['Sex', 'Embarked'])])),
                ('mode', RandomForestRegressor(random_state=0))])

In [130]:
red = pipe.predict(X_valid)

In [131]:
mean_absolute_error(y_valid, red)

0.2403320671865448

In [132]:
prediction = pipe.predict(test)

In [133]:
prediction

array([0.1125    , 0.23      , 0.02      , 0.7       , 0.42      ,
       0.4052381 , 0.33      , 0.14      , 0.76      , 0.04      ,
       0.        , 0.10566667, 1.        , 0.01      , 1.        ,
       0.93133333, 0.09242857, 0.62      , 0.55      , 0.21      ,
       0.23      , 0.73666667, 1.        , 0.1       , 0.9       ,
       0.        , 1.        , 0.65      , 0.55333333, 0.53      ,
       0.02      , 0.08      , 0.69      , 0.17      , 0.63083333,
       0.21666667, 0.12      , 0.26      , 0.02      , 0.44866667,
       0.23166667, 0.474     , 0.21916667, 1.        , 0.99      ,
       0.03      , 0.41733333, 0.12613364, 0.98      , 0.63      ,
       0.4       , 0.4       , 0.91      , 0.93      , 0.69      ,
       0.        , 0.        , 0.        , 0.01      , 0.99      ,
       0.05833333, 0.47166667, 0.        , 1.        , 0.8       ,
       0.86      , 0.92      , 0.01      , 0.165     , 0.89      ,
       0.99      , 0.        , 0.18      , 0.61      , 0.99   

In [ ]:
output = pd.DataFrame({'PassengerID': test.index,
                       'Survived': prediction})

In [ ]:
output.to_csv('Submission.csv', index=False)